In [1]:
import warnings
warnings.filterwarnings(action='ignore',
                        category=DeprecationWarning,
                        module='stable_baselines')
warnings.filterwarnings(action='ignore',
                        category=UserWarning,
                        module='stable_baselines')
warnings.filterwarnings("ignore", category=FutureWarning, module='tensorflow')
warnings.filterwarnings("ignore", category=FutureWarning, module='tensorboard')
warnings.filterwarnings("ignore", category=UserWarning, module='gym')

import gym
import PortfolioAllocationGym
import numpy as np
from stable_baselines import A2C
from stable_baselines.common.policies import MlpLnLstmPolicy #, MlpPolicy, MlpLstmPolicy
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.env_checker import check_env
from stable_baselines.bench import Monitor
from tensorflow import nn as nn

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from datetime import datetime

In [16]:
observations = ['daily_returns', 'ema_50', 'ema_200', 'bb_bbm', 'bb_bbh', 'bb_bbl','bb_bbhi', 'bb_bbli', 'stoch', 'stoch_signal', 'macd','macd_signal', 'obv']
env_kwargs = {'filename':'sp500.csv',
    'date_from':'2008-01-01',
    'date_to':'2017-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':10,
    'random_sample':True,
    'observations' : observations,
    'save_info' : True,
    #'report_point' : 252,
    'reward_function':'sharpe'}

train_env = gym.make('PortfolioAllocation-v0', **env_kwargs)
train_env = Monitor(train_env, 'monitor')

In [17]:
check_env(train_env)

In [18]:
venv, obs = train_env.get_sb_env()

In [19]:
#{'gamma': 0.9999, 'n_steps': 1, 'lr_schedule': 'constant', 'lr': 0.001, 'ent_coef': 0.1, 'vf_coef': 0, 'max_grad_norm': 5, 'n_lstm': 128, 'activation_fn': 'tanh', 'net_arch': 'medium'}.
model_kwargs =   {
    'gamma': 0.9999,
    'n_steps': 1,
    'lr_schedule': 'constant',
    'learning_rate': 0.001,
    'ent_coef': 0.1,
    'vf_coef': 0,
    'max_grad_norm': 5,
    'policy_kwargs' : dict (
        n_lstm=128,
        act_fun=nn.tanh,
        net_arch=[64, 'lstm', dict(pi=[256, 256], vf=[256, 256])]
        )
    }

a2c_model = A2C(policy = MlpLnLstmPolicy, tensorboard_log="tensorboard",env = venv, **model_kwargs)

#a2c_model_default = A2C(policy = MlpLnLstmPolicy, tensorboard_log="tensorboard",env = venv)

In [6]:
#a2c_model.get_parameter_list()

In [7]:
#a2c_model_default.get_parameter_list()

In [20]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


day: 2516                 reward: 4.697                 daily_returns: 10.018                 benchmark: 6.731                 index : 18.340                 sharpe: 15.676                 cum. rtns: 891.032                 portf val: 9,910,316.42
day: 2516                 reward: 3.932                 daily_returns: 7.565                 benchmark: 4.277                 index : 18.340                 sharpe: 9.037                 cum. rtns: 432.194                 portf val: 5,321,940.58
day: 2516                 reward: 1.450                 daily_returns: 5.089                 benchmark: 1.801                 index : 18.340                 sharpe: 4.504                 cum. rtns: 157.828                 portf val: 2,578,276.56
day: 2516                 reward: 2.044                 daily_returns: 6.011                 benchmark: 2.724                 index : 18.340                 sharpe: 5.861                 cum. rtns: 74.595                 portf val: 1,745,951.01
day: 2516      

In [21]:
total_timesteps = 20* (len(venv.venv.envs[0].data.date.unique()))

In [22]:
#total_timesteps = 500 * (len(venv.data.date.unique())-1)
trained_a2c_model= a2c_model.learn(total_timesteps=total_timesteps, #callback=tbc.TensorBoardCallback(),
                                   tb_log_name='A2C_lstm_bmk'+datetime.now().strftime("%H-%M"))

day: 2516                 reward: 1.913                 daily_returns: 3.803                 benchmark: 0.516                 index : 18.340                 sharpe: 4.836                 cum. rtns: 122.593                 portf val: 2,225,934.75
day: 2516                 reward: 1.804                 daily_returns: 5.249                 benchmark: 1.961                 index : 18.340                 sharpe: 6.189                 cum. rtns: 189.435                 portf val: 2,894,345.39
day: 2516                 reward: 2.107                 daily_returns: 6.209                 benchmark: 2.922                 index : 18.340                 sharpe: 7.904                 cum. rtns: 237.361                 portf val: 3,373,608.54
day: 2516                 reward: 3.049                 daily_returns: 6.425                 benchmark: 3.137                 index : 18.340                 sharpe: 8.403                 cum. rtns: 292.870                 portf val: 3,928,701.05
day: 2516       

In [23]:
trained_a2c_model.save('ac2_mlplnltsm_100_10_t18_random_sharpe.zip')

In [5]:
#trained_a2c_model =  A2C.load('ac2_mlplnltsm_100_10_t18.zip')





Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [12]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 reward: 1.081                 daily_returns: 4.368                 benchmark: 1.081                 index : 18.340                 sharpe: 4.913                 cum. rtns: 68.759                 portf val: 1,687,590.60
day: 2516                 reward: 1.081                 daily_returns: 4.368                 benchmark: 1.081                 index : 18.340                 sharpe: 4.913                 cum. rtns: 68.759                 portf val: 1,687,590.60
day: 2516                 reward: 1.081                 daily_returns: 4.368                 benchmark: 1.081                 index : 18.340                 sharpe: 4.913                 cum. rtns: 68.759                 portf val: 1,687,590.60
day: 2516                 reward: 1.081                 daily_returns: 4.368                 benchmark: 1.081                 index : 18.340                 sharpe: 4.913                 cum. rtns: 68.759                 portf val: 1,687,590.60
day: 2516           

In [24]:
eval_kwargs = {'filename':'sp500.csv',
    'date_from':'2018-01-01',
    'date_to':'2020-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':10,
    'random_sample':False,
    'observations' : observations,
    'save_info' : True,
    #'report_point' : 252,
    'reward_function':'sharpe'}

eval_env =  gym.make('PortfolioAllocation-v0', **eval_kwargs)

In [25]:
eval_venv, obs = eval_env.get_sb_env()


In [26]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, eval_venv, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


day: 653                 reward: 1.780                 daily_returns: 7.322                 benchmark: 2.841                 index : 64.277                 sharpe: 3.266                 cum. rtns: 48.570                 portf val: 1,485,697.19
day: 653                 reward: 2.987                 daily_returns: 9.366                 benchmark: 4.885                 index : 64.277                 sharpe: 4.234                 cum. rtns: 68.884                 portf val: 1,688,844.97
day: 653                 reward: 3.211                 daily_returns: 9.820                 benchmark: 5.339                 index : 64.277                 sharpe: 4.448                 cum. rtns: 73.977                 portf val: 1,739,767.77
day: 653                 reward: 3.226                 daily_returns: 9.842                 benchmark: 5.361                 index : 64.277                 sharpe: 4.464                 cum. rtns: 74.265                 portf val: 1,742,654.27
day: 653                